In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall -y tensorflow --quiet
!pip install git+https://github.com/ludwig-ai/ludwig.git@master --quiet
!pip install "git+https://github.com/ludwig-ai/ludwig.git@master#egg=ludwig[llm]" --quiet
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 53.7 MB/s eta 0:00

In [3]:
from ludwig.api import LudwigModel
import pandas as pd

In [4]:
model_dir = "/content/drive/MyDrive/Bioinformatics_401_Files/Ludwig_Model_Epoch_0/model"
ludwig_model = LudwigModel.load(model_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
!pip install jaro-winkler
!pip install scikit-learn
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.8 MB/s eta 0:00:00


In [14]:
import torch
from torcheval.metrics.functional import binary_f1_score
from sklearn.metrics import precision_score, recall_score
import json
from jaro import jaro_winkler_metric

def read_lines(file: str) -> list[str]:
    """Read lines from a file and return as a list."""
    with open(file, "r") as f:
        lines = f.readlines()
    return lines

def read_jsonl(file: str) -> list[list[tuple[str]]]:
    """Read JSON lines from a file and parse into a list of lists of tuples."""
    lines = read_lines(file)
    result = [[tuple(triplet) for triplet in json.loads(line)] for line in lines]
    return result

def read_txt(file: str) -> list[str]:
    """Read lines from a text file and return as a list."""
    return read_lines(file)

def to_binary(preds, gt):
    """Convert prediction and ground truth samples to binary classification."""
    max_len = max(len(preds), len(gt))
    output = [1 if pred in gt else 0 for pred in preds]
    target = [1 if gt in preds else 0 for gt in gt]

    # Pad the shorter tensor with zeros
    if len(output) < max_len:
        output.extend([0] * (max_len - len(output)))
    if len(target) < max_len:
        target.extend([0] * (max_len - len(target)))

    return torch.tensor(output), torch.tensor(target)

def calculate_scores(preds, gt):
    """Calculate F1 score, precision, recall, and Jaro-Winkler score."""
    max_len = max(len(gt), len(preds))
    f1_score_sum = precision_sum = recall_sum = jw_score_sum = 0

    for i in range(max_len):
        # Calculate F1 score, precision, and recall
        pred_sample = preds[min(i, len(preds) - 1)]
        gt_sample = gt[min(i, len(gt) - 1)]

        # Convert samples to lowercase
        pred_sample = [[item.lower() for item in triplet] for triplet in pred_sample]
        gt_sample = [[item.lower() for item in triplet] for triplet in gt_sample]

        output, target = to_binary(pred_sample, gt_sample)
        output, target = torch.tensor(output), torch.tensor(target)
        f1_score_sum += binary_f1_score(output, target)
        precision_sum += precision_score(target, output)
        recall_sum += recall_score(target, output)

        # Calculate Jaro-Winkler score
        pred_sample_txt = ' '.join([' '.join(triplet) for triplet in pred_sample])
        gt_sample_txt = ' '.join([' '.join(triplet) for triplet in gt_sample])
        jw_score_sum += jaro_winkler_metric(pred_sample_txt, gt_sample_txt)

    # Calculate averages
    f1_score = f1_score_sum / max_len
    precision = precision_sum / max_len
    recall = recall_sum / max_len
    jw_score = jw_score_sum / max_len

    return float(f1_score), precision, recall, jw_score

In [18]:
# Inport test set
import json
import ast
import statistics
from tqdm import tqdm

# Create lists
f1_score_list = []
precision_list = []
recall_list = []
jw_score_list = []
successful_output = 0
failed_output = 0

test_set_path = 'test_10.jsonl'
output_file_path = 'output.jsonl'
with open(test_set_path, 'r') as infile, open(output_file_path, "w") as outfile:
  for line in tqdm(infile):
    data = json.loads(line)
    abstract = data['input']
    ground_truth = data['output']

    # Create input
    test_examples = pd.DataFrame([
    {
        "instruction" : """
          The task is to extract chemical-related triples from scientific research papers.
          The rules are:
          1. Only use the following predicates in the triple: “causes”, ”biolocation is”, “exposed through”, “sourced through”, “has role of”, “involved in”.
          2. If there is more than one noun in the object, separate it into multiple triples.
          3. If you don't find relevant chemical-related triples in the paper or you are not sure, return: null.
          4. The response is an array of the relevant triples in the form: [subject, predicate, object].
          Q: Interaction of TMPC with ANXA2 mediated attachment and colonization of S. anginosus and induced mitogen-activated protein kinase (MAPK) activation.
          A: ["TMPC", "involved in", "MAPK activation"],
          ["ANXA2", "involved in", "MAPK activation"]
          Q: α-Lipoic acid plays an essential role in mitochondrial dehydrogenase reactions.
          A: ["alpha-Lipoic acid", "involved in", "mitochondrial dehydrogenase reactions"]
          Q: Ferroptosis, a form of regulated cell death that is driven by iron-dependent phospholipid peroxidation, has been implicated in multiple diseases, including cancer
          A: ["Ferroptosis", "causes", "cancer"]
          Q: These transporters and other biotin-binding proteins partition biotin to the cytoplasm and mitochondria cell compartments.
          A: ["Biotin", "biolocation is", "cytoplasm"],
          ["Biotin", "biolocation is", "mitochondria"]""",
        "input": abstract,
    }])

    # Run model
    predictions = ludwig_model.predict(test_examples)[0]
    output = predictions['output_response'][0][0]

    # Use ast.literal_eval() to convert the string to a list
    try:
        # Turn output to a list
        my_list = ast.literal_eval(output)
        gt = [[tuple(triplet) for triplet in ground_truth]]
        preds = [[tuple(triplet) for triplet in my_list]]

        # Get scores
        f1_score, precision, recall, jw_score = calculate_scores(preds, gt)

        # Print outputs
        print("\n\n")
        print(f"Ground Truth: {gt}")
        print(f"Output:       {preds}")
        print(f1_score, precision, recall, jw_score)

        # Increment variables
        f1_score_list.append(f1_score)
        precision_list.append(precision)
        recall_list.append(recall)
        jw_score_list.append(jw_score)
        successful_output += 1

    except (SyntaxError, ValueError) as e:
        failed_output += 1
        print(output)

    # Save the {'input': abstract, 'output': output} pair to JSONL
    json.dump({'input': abstract, 'output': output}, outfile)
    outfile.write('\n')

# Calculate statistics for each list
print("\n\n\n")
for name, score_list in [("F1 Score", f1_score_list), ("Precision", precision_list), ("Recall", recall_list), ("Jaro-Winkler Score", jw_score_list)]:
    avg_score = sum(score_list) / successful_output
    min_score = min(score_list)
    max_score = max(score_list)
    std_dev = statistics.stdev(score_list) if len(score_list) > 1 else 0  # Std. dev. requires at least 2 data points

    # Print statistics
    print(f"{name}:")
    print(f"  Average: {avg_score}")
    print(f"  Minimum: {min_score}")
    print(f"  Maximum: {max_score}")
    print(f"  Standard Deviation: {std_dev}")

# Print other information
total_outputs = successful_output + failed_output
print(f"Successful Outputs: {successful_output}")
print(f"Failed Outputs: {failed_output}")
print(f"Total Outputs: {total_outputs}")


0it [00:00, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
<ipython-input-14-cab98bc76cdf>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), r




Ground Truth: [[('(5Z)-(15S)-11-α-hydroxy-9,15-dioxoprosta-13-enoate', 'sourced through', 'Green tea'), ('Cannogenol 3-[glucosyl-(1->4)-2,6-dideoxy-xylohexoside]', 'sourced through', 'Green tea'), ('cis-Zeatin-7-N-glucoside', 'sourced through', 'Green tea'), ('Vinaginsenoside R7', 'sourced through', 'Green tea'), ('(R)-lipoate', 'sourced through', 'Green tea'), ('Testosterone', 'sourced through', 'Green tea')]]
Outpu:        [[('(5Z)-(15S)-11-α-hydroxy-9,15-dioxoprosta-13-enoate', 'sourced through', 'Green tea'), ('Cannogenol 3-[glucosyl-(1->4)-2,6-dideoxy-xylohexoside]', 'sourced through', 'Green tea'), ('cis-Zeatin-7-N-glucoside', 'sourced through', 'Green tea'), ('Vinaginsenoside R7', 'sourced through', 'Green tea'), ('(R)-lipoate', 'sourced through', 'Green tea'), ('Testosterone', 'sourced through', 'Green tea')]]
1.0 1.0 1.0 1.0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
<ipython-input-14-cab98bc76cdf>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, target = torch.tensor(output), torch.tensor(target)
2it [01:07, 34.84s/it]




Ground Truth: [[('cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(i-12:0/i-21:0/i-21:0/i-22:0)'), ('cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(16:0/20:4(5Z,8Z,11Z,14Z)/22:5(7Z,10Z,13Z,16Z,19Z)/16:0)'), ('cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(i-13:0/i-22:0/i-14:0/a-13:0)'), ('cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(14:0/15:0/16:0/20:0)'), ('cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(22:5(7Z,10Z,13Z,16Z,19Z)/20:4(5Z,8Z,11Z,14Z)/22:5(7Z,10Z,13Z,16Z,19Z)/22:5(7Z,10Z,13Z,16Z,19Z))')]]
Outpu:        [[('Cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(i-12:0/i-21:0/i-21:0/i-22:0)'), ('Cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(16:0/20:4(5Z,8Z,11Z,14Z)/22:5(7Z,10Z,13Z,16Z,19Z)/16:0)'), ('Cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(i-13:0/i-22:0/i-14:0/a-13:0)'), ('Cardiolipin', 'involved in', 'Cardiolipin Biosynthesis CL(14:0/15:0/16:0/20:0)'), ('Cardiolipin', 'involved in', 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
<ipython-input-14-cab98bc76cdf>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t

...




Ground Truth: [[('Momordenol', 'involved in', 'Lipid metabolism'), ('Glabranin', 'involved in', 'Lipid metabolism'), ('cis-6-Nitro-p-mentha-1(7),2-diene', 'involved in', 'Lipid metabolism'), ('D-Leucic acid', 'involved in', 'Lipid metabolism')]]
Outpu:        [[('Momordenol', 'involved in', 'Lipid Metabolism'), ('Glabranin', 'involved in', 'Lipid Metabolism'), ('cis-6-Nitro-p-mentha-1(7),2-diene', 'involved in', 'Lipid Metabolism'), ('D-Leucic acid', 'involved in', 'Lipid Metabolism')]]
1.0 1.0 1.0 1.0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
<ipython-input-14-cab98bc76cdf>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t




Ground Truth: [[('(11Z,14Z)-3-Icosa-11,14-dienoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('7-Hydroxyoctadecanoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('Pentadeca-5,12-dienoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('12-(3,4-Dimethyl-5-propylfuran-2-yl)dodecanoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('3-Hydroxytetradeca-4,7,10-trienoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('15-Methyltricosanoylcarnitine', 'causes', 'Inflammatory bowel disease')]]
Outpu:        [[('(11Z,14Z)-3-Icosa-11,14-dienoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('7-Hydroxyoctadecanoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('Pentadeca-5,12-dienoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('12-(3,4-Dimethyl-5-propylfuran-2-yl)dodecanoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('3-Hydroxytetradeca-4,7,10-trienoylcarnitine', 'causes', 'Inflammatory bowel disease'), ('15-Methyltricosanoylcarnitine

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
<ipython-input-14-cab98bc76cdf>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t




Ground Truth: [[('adenylyl-molybdopterin', 'sourced through', 'Other soy product'), ('L-Ascorbic acid', 'sourced through', 'Other soy product'), ('(Z)-3-Hexenal', 'sourced through', 'Other soy product'), ('(24R)-Ergost-4-en-3-one', 'sourced through', 'Other soy product'), ('D-sorbitol', 'sourced through', 'Other soy product')]]
Outpu:        [[('TMPC', 'involved in', 'MAPK activation'), ('ANXA2', 'involved in', 'MAPK activation'), ('Ferroptosis', 'causes', 'cancer'), ('Biotin', 'involved in', 'mitochondrial dehydrogenase reactions'), ('Biotin', 'involved in', 'MAPK activation')]]
0.0 0.0 0.0 0.5677751399862995




F1 Score:
  Average: 0.7777777777777778
  Minimum: 0.0
  Maximum: 1.0
  Standard Deviation: 0.44095855184409843
Precision:
  Average: 0.7777777777777778
  Minimum: 0.0
  Maximum: 1.0
  Standard Deviation: 0.44095855184409843
Recall:
  Average: 0.7777777777777778
  Minimum: 0.0
  Maximum: 1.0
  Standard Deviation: 0.44095855184409843
Jaro-Winkler Score:
  Average: 0.8972829